# Summary
In this notebook we will setup a *TSEB* model to run a time series data at point scale.

# Select input and output tables
The main input data consists on an ASCII table with tab separators and the firs row containing the name of the variables. At least the table should contain the following field names (case sensitive):
>* **TSEB-PT** : `Year, DOY, Time, Trad, VZA, Ta, u, ea, Sdn, LAI & hc`
>* **DTD** : `Year, DOY, Time, Trad_0, Trad_1, VZA, Ta_0, Ta_1, u, ea, Sdn, LAI & hc`
>* **TSEB-2T** : `Year, DOY, Time, Tc, Ts, Ta, u, ea, Sdn, LAI & hc`

**Year** is not really used in the model but just kept in case several years were processed, **DOY** is the day of the year (1-366), **Time** is the measurement time in decimal hours (0-24). **Trad** is the radiometric composite temperature; **Trad_0** and **Trad_1**, the same for time 0 (near sunrise0 and time 1 (around noon) for *DTD* model: **Ts** and **Tc** are the component soil and canopy temperatures for *TSEB-2T*; **Ta** is the air temperature: all temperatures expressed in Celsius. **u** is the wind speed measurement (m s$^{-1}$), **ea** is the atmospheric vapour pressure (mb), **Sdn** is the shortwave irradiance (W m$^{-2}$), **LAI** is the effective leaf area index, and **hc** is the canopy height (m).

Additional input variables that can be included in the table are: **SAA** solar azimuth angle, **SZA** solar zenith angle, **Ldn** incoming longwave radiation (W m$^{-2}$), **f_C** fractional cover, **f_g** green fraction, **wc** canopy width to height ratio, or **G'** soil heat flux (W m$^{-2}$)

If any of those additional variables are not found in the table they will be internally estimated by TSEB or use default values. The order of the columns is not relevant, and neither whethere there are additional columns in the table (they will be ignored if their names do not match any of the possible input variables.

## TSEB setup
Select the input and output filles and then fill all the information in each tab:

In [ ]:
from pyTSEB import PyTSEB # import the PyTSEB class object in the pyTSEB.py module
setup=PyTSEB() # Create the setup instance from the PyTSEB class object
setup.PointTimeSeriesWidget() # Load the TSEB configuration Widget

In [ ]:
# Get the data from the widgets
setup.GetDataTSEBWidgets(False)
print(setup.lat,setup.lon,setup.alt,setup.stdlon,setup.z_u,setup.z_t)
#print(setup.TSEB_MODEL)
#print(setup.emisVeg,setup.emisGrd)
#print(setup.spectraVeg,setup.spectraGrd)
print(setup.Max_alpha_PT, setup.x_LAD, setup.leaf_width,setup.z0_soil,setup.LANDCOVER)
#print(setup.KN_b,setup.KN_c,setup.KN_C_prime)
#print(setup.CalcG)

In [ ]:
setup.RunTSEBPointSeries()

* [Click here for know more about the TSEB model](./TSEB_and_Resistances.ipynb "The TSEB model")
* [To know more about the TSEB code press here](./pyTSEB_in_Detail.ipynb "Introduction to TSEB")